# Generative AI Basics

### 🎯 Objective
This notebook introduces **Generative AI (GenAI)** : a field of machine learning where models learn to **generate new data** similar to what they were trained on.

By the end of this notebook, you will understand:
- What generative AI is and how it differs from discriminative AI.
- Major types of generative models (VAE, GANs, Diffusion models, Transformers).
- How to build a simple **Variational Autoencoder (VAE)** to generate new samples.
- Key evaluation metrics for generative models.

## 🧠 1. What is Generative AI?

**Generative AI** refers to systems capable of creating new data : such as text, images, music, or code — that mimic human creativity.

In contrast to **discriminative models** (which learn decision boundaries), **generative models** learn the **data distribution** $P(X)$ or $P(X|Y)$.

### Examples:
- 🖼️ Image Generation → *Stable Diffusion, DALL·E, Midjourney*
- 📝 Text Generation → *GPT, Claude, Gemini*
- 🎵 Audio/Music Generation → *Jukebox, MusicLM*
- 🎥 Video Generation → *Sora, Runway*

Generative AI is the foundation of most modern **creative AI applications**.

## ⚙️ 2. Types of Generative Models

| Model Type | Core Idea | Example |
|-------------|------------|----------|
| **Autoencoders (AE)** | Learn compressed representations (latent space) | Denoising AE |
| **Variational Autoencoders (VAE)** | Learn probability distribution in latent space | Image synthesis |
| **Generative Adversarial Networks (GANs)** | Generator vs Discriminator game | DeepFake, StyleGAN |
| **Diffusion Models** | Learn to reverse noise to data | Stable Diffusion |
| **Transformers (LLMs)** | Sequence prediction-based generation | GPT, BERT, etc. |

## 🧩 3. Variational Autoencoder (VAE)

A **VAE** is a probabilistic generative model that learns to map data into a latent space, from which new samples can be generated.

### VAE Structure:
- **Encoder:** Maps input → mean (μ) and variance (σ²) of latent distribution.
- **Reparameterization Trick:** Sample z = μ + σ * ε, where ε ~ N(0,1).
- **Decoder:** Reconstructs data from latent variable z.

![VAE Diagram](https://developers.google.com/machine-learning/gan/images/vae-diagram.svg)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

class VAE(nn.Module):
    def __init__(self, latent_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc_mu = nn.Linear(400, latent_dim)
        self.fc_logvar = nn.Linear(400, latent_dim)
        self.fc_decode = nn.Linear(latent_dim, 400)
        self.fc_out = nn.Linear(400, 784)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc_mu(h), self.fc_logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc_decode(z))
        return torch.sigmoid(self.fc_out(h))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

vae = VAE()
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

### 🧮 VAE Loss Function
The loss combines two terms:
1. **Reconstruction Loss** : how well the output matches the input.
2. **KL Divergence Loss** : how close the latent distribution is to a standard normal distribution.

$$ \mathcal{L} = \text{ReconstructionLoss} + KL( q(z|x) || p(z) ) $$

In [ ]:
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

# Training loop (simplified)
vae.train()
for epoch in range(2):  # fewer epochs for demo
    total_loss = 0
    for data, _ in train_loader:
        data = data.view(-1, 784)
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader.dataset):.4f}")

### 🎨 Generate New Samples
Once trained, we can sample random vectors from the latent space to generate new digits.

In [ ]:
import matplotlib.pyplot as plt

vae.eval()
with torch.no_grad():
    z = torch.randn(16, 20)
    samples = vae.decode(z).view(-1, 1, 28, 28)

grid = torch.cat([torch.cat([samples[i*4 + j] for j in range(4)], dim=2) for i in range(4)], dim=1)
plt.imshow(grid.squeeze().cpu().numpy(), cmap='gray')
plt.axis('off')
plt.title('Generated Digits by VAE')
plt.show()

## 🤖 4. Beyond VAEs — Other Generative Models

### 🔹 Generative Adversarial Networks (GANs)
- Two networks — **Generator** and **Discriminator** — compete.
- Generator learns to fool the discriminator by generating realistic samples.
- Popular in **image synthesis, upscaling, and deepfakes.**

### 🔹 Diffusion Models
- Learn to reverse a process of adding noise step-by-step.
- Used in **DALL·E 2**, **Stable Diffusion**, **Imagen**.

### 🔹 Large Language Models (LLMs)
- Predict the next token given context.
- Used in **ChatGPT, Claude, Gemini**.

### Comparison
| Model | Pros | Cons |
|--------|------|------|
| VAE | Stable training, interpretable latent space | Blurry images |
| GAN | Sharp outputs, realistic | Unstable training |
| Diffusion | High fidelity | Slow inference |
| Transformers | Great for sequences | Needs large data |

## 🧾 5. Evaluating Generative Models
Evaluating generative quality is subjective, but key metrics include:
- **Inception Score (IS):** Measures how realistic and diverse images are.
- **Fréchet Inception Distance (FID):** Measures distance between generated and real image distributions.
- **Perplexity:** Used for text generation.
- **Human evaluation:** Gold standard for creativity.

## ✅ Summary
- **Generative AI** focuses on creating new data, unlike discriminative models that classify existing data.
- **VAEs** learn latent distributions; **GANs** play an adversarial game; **Diffusion models** denoise step-by-step.
- These models form the foundation of modern generative tools powering **AI art, chatbots, and creativity apps**.

🚀 **Next:** Try exploring `GAN_Basics.ipynb` or `Diffusion_Models.ipynb` to deepen your understanding of creative AI generation.